In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
filename= "../data/kobe/kobe_bryant_shot_data.csv.gz"
df = pd.read_csv(filename, na_values={'shot_made_flag': ''})
df = df.dropna()
df = df.drop([u'game_event_id', u'game_id', 'combined_shot_type',
       u'lat', u'lon', u'team_id', u'team_name', u'game_date',
        u'shot_id'], axis=1)

df = df.drop(['loc_x', 'loc_y', 'shot_type','shot_zone_basic', 'shot_zone_range'], axis=1)

In [3]:
df['home'] = df.matchup.apply(lambda matchup: 0 if '@' in matchup else 1)
df = df.drop(['matchup'], axis=1)

df['time_remaining'] = 60 * df['minutes_remaining'] + df['seconds_remaining']
df = df.drop(['minutes_remaining', 'seconds_remaining'], axis=1)

cols = df.columns.tolist()
cols.remove('shot_made_flag')
cols.append('shot_made_flag')

df = df[cols]

In [4]:
filename= "../data/kobe/kobe_bryant_shot_data_refined.csv"
df.to_csv(filename, index=False)

In [5]:
filename= "../data/kobe/kobe_bryant_shot_data_refined.csv"
df = pd.read_csv(filename)

In [6]:
df

,action_type,period,playoffs,season,shot_distance,shot_zone_area,opponent,home,time_remaining,shot_made_flag
0,Jump Shot,1,0,2000-01,15,Left Side(L),POR,0,622,0.0
1,Jump Shot,1,0,2000-01,16,Left Side Center(LC),POR,0,465,1.0
2,Jump Shot,1,0,2000-01,22,Right Side Center(RC),POR,0,412,0.0
3,Driving Dunk Shot,2,0,2000-01,0,Center(C),POR,0,379,1.0
4,Jump Shot,3,0,2000-01,14,Left Side(L),POR,0,572,0.0
5,Layup Shot,3,0,2000-01,0,Center(C),POR,0,532,1.0
6,Jump Shot,3,0,2000-01,12,Left Side(L),POR,0,372,1.0
7,Running Jump Shot,3,0,2000-01,12,Center(C),POR,0,216,0.0
8,Jump Shot,3,0,2000-01,25,Left Side Center(LC),POR,0,116,0.0
9,Jump Shot,1,0,2000-01,17,Right Side Center(RC),UTA,1,660,1.0


In [7]:
original_df = df.copy()

In [8]:
# turn categorical variables into dummy variables
categorical_vars = ['season', 'period', 'shot_zone_area', 'opponent', 'action_type']
for var in categorical_vars:
    df = pd.concat([df, pd.get_dummies(df[var], prefix=var)], 1)
    df = df.drop(var, 1)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.cross_validation import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('shot_made_flag', axis=1), 
                                                    df['shot_made_flag'], 
                                                    test_size=0.33, 
                                                    random_state=42)

In [12]:
model = RandomForestClassifier(n_estimators=45, max_depth=14, criterion='entropy', random_state=42, n_jobs=-1)

In [13]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=14, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=45, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [14]:
y_pred = model.predict(X_test)

In [15]:
y_pred_proba = model.predict_proba(X_test)

In [16]:
confusion_matrix(y_test, y_pred), log_loss(y_test, y_pred_proba[:,1])

(array([[3982,  716],
        [2013, 1770]]), 0.61507677659290805)

In [17]:
pd.DataFrame({'feature': X_train.columns, 
              'importance': model.feature_importances_}).sort_values('importance', ascending=False).head()

,feature,importance
95,action_type_Jump Shot,0.216242
1,shot_distance,0.108848
3,time_remaining,0.088063
96,action_type_Layup Shot,0.051611
116,action_type_Slam Dunk Shot,0.046677


In [18]:
pred_df = original_df.join(pd.DataFrame(y_pred, columns=['shot_made_pred'], index=X_test.index))

In [19]:
pred_df = pred_df[~pred_df.shot_made_pred.isnull()]

In [20]:
pred_df.head()

,action_type,period,playoffs,season,shot_distance,shot_zone_area,opponent,home,time_remaining,shot_made_flag,shot_made_pred
3,Driving Dunk Shot,2,0,2000-01,0,Center(C),POR,0,379,1.0,1.0
6,Jump Shot,3,0,2000-01,12,Left Side(L),POR,0,372,1.0,0.0
17,Jump Shot,3,0,2000-01,18,Right Side Center(RC),UTA,1,544,0.0,0.0
29,Jump Shot,2,0,2000-01,27,Center(C),VAN,0,4,1.0,0.0
30,Jump Shot,3,0,2000-01,18,Left Side(L),VAN,0,672,0.0,0.0


In [21]:
pred_df[(pred_df.shot_made_flag != pred_df.shot_made_pred)]

,action_type,period,playoffs,season,shot_distance,shot_zone_area,opponent,home,time_remaining,shot_made_flag,shot_made_pred
6,Jump Shot,3,0,2000-01,12,Left Side(L),POR,0,372,1.0,0.0
29,Jump Shot,2,0,2000-01,27,Center(C),VAN,0,4,1.0,0.0
35,Reverse Dunk Shot,3,0,2000-01,0,Center(C),VAN,0,175,0.0,1.0
49,Driving Layup Shot,3,0,2000-01,5,Center(C),LAC,1,37,0.0,1.0
54,Jump Shot,1,0,2000-01,25,Center(C),HOU,0,191,1.0,0.0
75,Jump Shot,2,0,2000-01,18,Left Side Center(LC),SAS,0,195,1.0,0.0
91,Driving Layup Shot,1,0,2000-01,0,Center(C),HOU,1,617,0.0,1.0
101,Jump Shot,3,0,2000-01,15,Right Side(R),HOU,1,518,1.0,0.0
102,Jump Shot,3,0,2000-01,15,Left Side(L),HOU,1,318,1.0,0.0
115,Driving Layup Shot,1,0,2000-01,0,Center(C),DEN,1,463,0.0,1.0


In [ ]:
# TODO: Other models
# Guess by prev shot? Is it even leagle?
# 'action_type' made a huge diff! Why?

In [22]:
# http://scikit-learn.org/stable/modules/ensemble.html#forests-of-randomized-trees

In [23]:
# http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

In [24]:
# http://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html

In [25]:
# http://scikit-learn.org/stable/auto_examples/svm/plot_iris.html#example-svm-plot-iris-py